In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.client import device_lib
from tensorflow.keras.layers import BatchNormalization
import numpy as np
import random

# シード値を固定
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# 🐱🐶 画像分類モデルの学習
print(Sequential)
# GPUがあるか確認して、あればメモリ制限
print(device_lib.list_local_devices())
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print("✅ GPUが見つかりました")
    try:
        # 動的メモリ割り当てを有効化
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPUメモリの動的割り当てを設定しました")
    except RuntimeError as e:
        print("❌ メモリ設定に失敗しました：", e)
        
# 🟡 ここに設定をまとめておくと便利！
IMG_WIDTH = 224
IMG_HEIGHT = 224
BATCH_SIZE =16
EPOCHS = 30 # 「全部の画像 × 3周」を学習
INPUT_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)  # 画像の縦・横・チャンネル数

train_dir = 'dataset/train'
val_dir = 'dataset/validation'

# 📁 データの前処理
# 訓練用ジェネレーター（オーグメンテーション付き）
# オーグメンテーション（回転やズームなど）をかけながら、ランダムに画像を取り出して学習させる
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1./255)

# ディレクトリから画像を読み込む
train_generator = train_datagen.flow_from_directory(
    train_dir,  # 訓練データのパス
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True,
    seed=seed_value
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False,
    seed=seed_value # 乱数シードを固定
    # shuffle=False, # シャッフルしない場合は、Trueにするとエラーになるので注意
)




In [ ]:
# 🧠 モデル構築
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# ResNet50のベースモデル（ImageNet学習済み）を使う
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # 最初は凍結して特徴抽出器として使う

# カスタム分類層を上に乗せる
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Flattenの代わりで軽量

x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)

x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)

x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)

x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)

predictions = Dense(1, activation='sigmoid')(x)


# モデル定義
model = Model(inputs=base_model.input, outputs=predictions)

# コンパイル
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("✅ ResNet50ベースのモデルが構築されました")



In [ ]:
# 🛠️ コンパイル

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("✅ ResNet50ベースのモデルが構築されました")

In [ ]:
# モデル学習
from tensorflow.keras.callbacks import EarlyStopping 
from tensorflow.keras.callbacks import ReduceLROnPlateau


early = EarlyStopping(
    monitor='val_loss',   # 何を監視するか
    patience=5,           # 何エポック連続で改善しなければ止めるか
    restore_best_weights=True,
    verbose=1)

reduce = ReduceLROnPlateau(
    monitor='val_loss',   # 監視対象
    factor=0.5,           # LR を 0.5 倍に
    patience=2,           # 2 エポック改善しなければ発動
    min_lr=1e-6,          # これ以下には下げない
    verbose=1)

try:
    # バッチごとに処理を試す最小限の設定
    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=val_generator,
        callbacks=[early, reduce],
        workers=1,
        use_multiprocessing=False
    )
    print("✅ モデル学習が正常に完了しました")
    # 学習成功時にモデル保存
    model.save('cat_dog_model.h5')
    print("モデルを cat_dog_model.h5 に保存しました")
    
except Exception as e:
    print(f"❌ エラーが発生しました: {str(e)}")
    model.save('cat_dog_model_error.h5')

In [ ]:
# 🧪 モデル評価
# "validation" フォルダの中にある画像を使って、モデルの評価を行うにゃ！
# ➊ 検証用ジェネレーターを作成（shuffle=False でファイル順に対応）
# 画像に手を加えずに、そのまま順番に取り出して評価する
val_datagen = ImageDataGenerator(rescale=1./255)
val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=1,
    class_mode='binary',
    shuffle=False,                  # 検証はシャッフルしない
    seed=seed_value 
)

# ➊ モデルの評価（検証データ全体での最終的な loss と accuracy を取得）
loss_eval, acc_eval = model.evaluate(val_gen, verbose=1)
print(f"▶ 最終検証損失: {loss_eval:.4f}")
print(f"▶ 最終検証精度: {acc_eval:.4f}")
print("▶ モデルの評価が完了しましたにゃ！")

from datetime import datetime
fname = f"cat_dog_{datetime.now():%Y%m%d_%H%M}_acc{acc_eval:.2f}.h5"
model.save(fname)
print(f"モデルを {fname} に保存しました🐾")

In [ ]:
# 📊 学習結果の可視化
import matplotlib.pyplot as plt
# フォント設定（例：Windows + メイリオ）
plt.rcParams['font.family'] = 'Meiryo'
plt.rcParams['axes.unicode_minus'] = False

# 学習過程の損失と精度をプロット
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(12, 4))

# 精度のグラフ
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='訓練精度')
plt.plot(epochs_range, val_acc, label='検証精度')
plt.legend(loc='lower right')
plt.title('精度の推移')

# 損失のグラフ
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='訓練損失')
plt.plot(epochs_range, val_loss, label='検証損失')
plt.legend(loc='upper right')
plt.title('損失の推移')

plt.show()

# 6. 予測結果を CSV に出力
import pandas as pd
preds       = model.predict(val_gen, steps=len(val_gen), verbose=1)
pred_labels = (preds >= 0.5).astype(int).flatten()
true_labels = val_gen.classes
filenames   = val_gen.filenames

df = pd.DataFrame({
    'filename': filenames,
    'true':     true_labels,
    'pred':     pred_labels
})
df.attrs['param'] = {'epoch':EPOCHS, 'batch':BATCH_SIZE,
                     'lr':1e-4, 'aug':'std'}
df['correct'] = df['true'] == df['pred']
df.to_csv('classification_results.csv', index=False)
print("▶ classification_results.csv に書き出しました")
